In [1]:
from anet_api.db import Team, Athlete, Meet, TeamCreate
from anet_api.db.utils import create_team
import requests as r
from sqlmodel import select, Session
from anet_api.db.database import SessionLocal
import json

In [2]:
api_url = 'http://127.0.0.1:8000/'

In [3]:
r.post(api_url + 'team/addTeam',data=json.dumps({'anet_id':493,'name':'Garfield'}))

<Response [400]>

In [4]:
def get_all_teams(session: Session):
    stmt = select(Team)
    return session().exec(stmt).all()

def insert_meets_and_athletes(params):
    q = r.get(api_url + 'team/getInfo', params = params)
    meets = q.json()['schedule']
    roster = q.json()['roster']
    for meet in meets:
        response = r.post(api_url + 'meet/addMeet',data=json.dumps(meet))
        if response.status_code != 200:
            print(meet['meet'], response.text)
    for athlete in roster:
        response = r.post(api_url + 'athlete/addAthlete', data=json.dumps(athlete))
        if response.status_code != 200:
            print(athlete['anet_id'], response.text)
    return

In [5]:
season = 2023
sport = 'xc'
for team in get_all_teams(SessionLocal):
    print(team)
    params = dict(team_id = team.anet_id, season = season, sport = sport)
    insert_meets_and_athletes(params)

last_edited=datetime.datetime(2024, 9, 27, 7, 36, 8, 467081) name='Garfield' anet_id=493 created_at=datetime.datetime(2024, 9, 27, 7, 34, 7, 649564) id=1
Steens Mountain Running Camp Jan Yriarte Uphill 5K  {"detail":"Meet already exists"}
Tahoma Coed Relays (Jamboree) {"detail":"Meet already exists"}
Metro XC @ Lincoln Park WS/Ballard/Garfield/CLeveland/Chief Sealth/East Side Catholic  {"detail":"Meet already exists"}
BROOKS Fort Steilacoom Invitational {"detail":"Meet already exists"}
[HOLD] Metro 3A League #2  {"detail":[{"loc":["body","venue"],"msg":"none is not an allowed value","type":"type_error.none.not_allowed"}]}
Metro XC @ Lower Woodland Cleveland/Chief Sealth/Garfield/Ballard/West Seattle  {"detail":"Meet already exists"}
27th John Payne XC Invitational {"detail":"Meet already exists"}
N.Hale Chief Sealth Garfield {"detail":"Meet already exists"}
3rd Annual Mountain Loop Invitational {"detail":"Meet already exists"}
Metro League XC Week #5 {"detail":"Meet already exists"}
20

In [6]:
from sqlalchemy import extract
def get_all_meets_by_season(session: Session, season: int):
    stmt = select(Meet).where(extract('year', Meet.date) == season)
    return session.exec(stmt).all()

In [7]:

meets = get_all_meets_by_season(SessionLocal(),2023)




In [8]:
meets

[Meet(venue='Steens Mountain ', city='FrenchGlen', zipcode=97736, id=2, anet_id=225111, last_edited=datetime.datetime(2024, 9, 27, 16, 37, 55, 16434), created_at=datetime.datetime(2024, 9, 27, 16, 34, 45, 579588), meet='Steens Mountain Running Camp Jan Yriarte Uphill 5K ', address=None, state='OR', date=datetime.date(2023, 7, 28)),
 Meet(venue='Lake Wilderness State Park', city='Maple Valley', zipcode=98038, id=3, anet_id=222038, last_edited=datetime.datetime(2024, 9, 27, 16, 37, 55, 25497), created_at=datetime.datetime(2024, 9, 27, 16, 34, 45, 579588), meet='Tahoma Coed Relays (Jamboree)', address='22500 SE 248th Street ', state='WA', date=datetime.date(2023, 9, 9)),
 Meet(venue='Lincoln Park', city='Seattle', zipcode=98136, id=4, anet_id=229674, last_edited=datetime.datetime(2024, 9, 27, 16, 37, 55, 32062), created_at=datetime.datetime(2024, 9, 27, 16, 34, 45, 579588), meet='Metro XC @ Lincoln Park WS/Ballard/Garfield/CLeveland/Chief Sealth/East Side Catholic ', address='8011 Fauntle

In [9]:
def insert_new_result(meet_id: int):
    params = dict(sport = 'xc', meet_id = meet_id)
    s = r.get(api_url + 'meet/getResults',params=params)
    s.json()

    all_results = []
    for race in s.json()['races']:
        all_results += race['results']

    for result in all_results:
        response = r.post(api_url + 'meet/addResult', data = json.dumps(result))
        if response.status_code == 422:
            print(response.text)
    return "All done!"

In [10]:
for meet in meets:
    insert_new_result(meet.anet_id)

{"detail":[{"loc":["body","anet_athlete_id"],"msg":"none is not an allowed value","type":"type_error.none.not_allowed"}]}
{"detail":[{"loc":["body","anet_athlete_id"],"msg":"none is not an allowed value","type":"type_error.none.not_allowed"}]}
{"detail":[{"loc":["body","anet_athlete_id"],"msg":"none is not an allowed value","type":"type_error.none.not_allowed"}]}
{"detail":[{"loc":["body","anet_athlete_id"],"msg":"none is not an allowed value","type":"type_error.none.not_allowed"}]}
{"detail":[{"loc":["body","anet_athlete_id"],"msg":"none is not an allowed value","type":"type_error.none.not_allowed"}]}
{"detail":[{"loc":["body","anet_athlete_id"],"msg":"none is not an allowed value","type":"type_error.none.not_allowed"}]}
{"detail":[{"loc":["body","anet_athlete_id"],"msg":"none is not an allowed value","type":"type_error.none.not_allowed"}]}
{"detail":[{"loc":["body","anet_athlete_id"],"msg":"none is not an allowed value","type":"type_error.none.not_allowed"}]}
{"detail":[{"loc":["body

In [26]:
session = SessionLocal()
team = Team(anet_id=111, name="New Team")
athlete = Athlete(anet_id=222, first_name="John", last_name="Doe", gender="F")
meet = Meet(anet_id=333, name="New Meet", venue="Place", date="2023-01-01")

session.add(team)
session.add(athlete)
session.add(meet)
session.commit()

In [11]:
import requests as r
url = 'https://www.athletic.net/api/v1/Event/Events'
data = dict(
        start='2022-11-01',
        end='2022-11-30',
        levelMask=4,
        sportMask=2,
        state='WA',
        country='us',
    )

res = r.post(url,json=data)

In [12]:
res.json()

{'events': [{'IDMeet': 224294,
   'Sport': 'XC',
   'MeetName': 'Forks HS League Meet (CANCELED)',
   'SeasonId': 2023,
   'StartDate': '0001-01-01T00:00:00',
   'EndDate': '2023-09-19T00:00:00',
   'Virtual': False,
   'LocationName': 'Olympic Natural Resources Center',
   'StreetAddress': '1455 S Forks Ave',
   'City': 'Forks',
   'PostalCode': '98331',
   'State': 'WA',
   'OwnerID': 364,
   'CalendarLock': 0,
   'UCalendarLock': 0,
   'RegEnd': None,
   'HasResults': 0,
   'LevelMask': 6,
   'HostName': 'Forks',
   'MascotUrl': '//lh3.googleusercontent.com/AUH2hZZ7KFJd1HnvbCxwUWO5zaQuN2YUpzeckUG9V9bd6Y6BpCO4ajPePXdkY0zjbVNaEfaKkhQsHPFr08_kMw',
   'Lat': 47.9330028,
   'Long': -124.3944981,
   'score': 0.0,
   'CalCount': 1,
   'OffDays': '{}',
   'Data': None,
   'rsUrl': None,
   'LiveID': None,
   'LivePublished': True,
   'VideoURL': None,
   'Website': '',
   'Country': 'US'},
  {'IDMeet': 225599,
   'Sport': 'XC',
   'MeetName': 'Removed',
   'SeasonId': 2023,
   'StartDate': 